In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('./dataexport3.csv')

In [2]:
data

,ProductLineID,PlanningPeriod,Material,Plant,PlannedPeriod,PeriodDiff,FieldID,Field,FieldContent
0,PCR,2022-01-01,AL19550015VAL4A00,5V01,2021-03-01,-10,ACTSLS,SalesYearMinus1,1
1,PCR,2022-01-01,AP21560017TC2SA00,5V24,2021-03-01,-10,ACTSLS,SalesYearMinus1,0
2,PCR,2022-01-01,AP27555020VHTRA02,5V04,2022-12-01,11,SLSBDG,SalesBudget,0
3,PCR,2022-01-01,AP26545021WWPPA02,5V06,2019-07-01,-30,ACTSLS,SalesYearMinus3,0
4,PCR,2022-01-01,AP25530019YUVPA02,5V10,2022-11-01,10,FNLBSFRC,FinalBaseForecast,0
...,...,...,...,...,...,...,...,...,...
2961499,PCR,2022-01-01,AP25555019WQT5A02,5V27,2021-03-01,-10,ACTSLS,SalesYearMinus1,0
2961500,PCR,2022-01-01,AP18565015TTT2A00,5V18,2021-02-01,-11,ACTSLS,SalesYearMinus1,0
2961501,PCR,2022-01-01,AP21555016VSP5A02,5V04,2022-06-01,5,SLSBDG,SalesBudget,0
2961502,PCR,2022-01-01,AP28545020WWPRA02,5V13,2022-12-01,11,FNLRLLFRC,FinalRollingForecast,4


In [44]:
# FIlter by Actual Sales
data = data.loc[data['FieldID'] == 'ACTSLS'] 
# Drop un-needed Columns
data.drop(['FieldID', 'Field', 'UpdateBy', 'UpdateWhen', 'PlanningPeriod', 'PeriodDiff'], axis=1, inplace=True)
# Rename Columns
data.rename(columns={'FieldContent':'Volume'}, inplace=True)
data.rename(columns={'PlannedPeriod':'Date'}, inplace=True)
cols = [
    'Plant',
    'Material',
    'Volume',
    'Date',
    'SalesTargetGroup',
    ]
# Rearrange Columns
data = data[cols]
# Date column to Datetime format
data['Date']= pd.to_datetime(data['Date'])
# Time Index from Dates
data["time_idx"] = data["Date"].dt.year * 12 + data["Date"].dt.month
data["time_idx"] -= data["time_idx"].min()
# Creating Month (Must be redone for Colab, maybe it dislikes being exported as CSV)
data["month"] = data.Date.dt.month.astype(str).astype("category")
# Set Volums lower than 0 to 0
data.Volume=data.Volume.mask(data.Volume.lt(0),0)
# Generate Columns
data["log_volume"] = np.log(data.Volume + 1e-8)
data["avg_volume_by_material"] = data.groupby(["time_idx", "Material"], observed=True).Volume.transform("mean")
data["avg_volume_by_plant"] = data.groupby(["time_idx", "Plant"], observed=True).Volume.transform("mean")
# Enumerate groups as timeseries 
data = data.assign(
    timeseries=pd.factorize(list(zip(
        *[data[c].values.tolist() for c in ['Plant', 'Material', 'SalesTargetGroup']]
    )))[0] + 1
)
# Remove NaN rows
data = data[data['Material'].notna()]

In [ ]:
data.groupby(["timeseries"], observed=True).Volume.transform("sum") != 0 

In [7]:
data

,Plant,Material,Volume,Date,SalesTargetGroup,time_idx,month,log_volume,avg_volume_by_material,avg_volume_by_plant,timeseries
0,5M01,AI15570013TARCA20,0,2019-02-01,510,0,2,-18.420681,0.894737,4.137127,1
1,5M01,AI15570013TARCA20,0,2019-03-01,510,1,3,-18.420681,2.631579,5.976697,1
2,5M01,AI15570013TARCA20,0,2019-04-01,510,2,4,-18.420681,0.000000,3.653115,1
3,5M01,AI15570013TARCA20,0,2019-05-01,510,3,5,-18.420681,0.000000,3.689889,1
4,5M01,AI15570013TARCA20,0,2019-06-01,510,4,6,-18.420681,0.421053,4.761486,1
...,...,...,...,...,...,...,...,...,...,...,...
17180971,5V21,TT38652264TT3RTA0,0,2021-09-01,5A4,31,9,-18.420681,0.000000,2.284550,204536
17180972,5V21,TT38652264TT3RTA0,0,2021-10-01,5A4,32,10,-18.420681,0.000000,2.233365,204536
17180973,5V21,TT38652264TT3RTA0,0,2021-11-01,5A4,33,11,-18.420681,0.000000,3.511848,204536
17180974,5V21,TT38652264TT3RTA0,0,2021-12-01,5A4,34,12,-18.420681,0.000000,1.894408,204536


In [45]:
data["time_idx"].max()

35

In [20]:
data.Volume=data.Volume.mask(data.Volume.lt(0),0)

In [9]:
def non_norm(data):
    timeseries_list= list(set(data["timeseries"]))
    non_norm = []
    for timeseries in timeseries_list:
        if len(data[data['timeseries'] == timeseries]) != 36:
            non_norm.append(timeseries)
    return non_norm

In [10]:
non_norm(data)

[]

In [46]:
data["Volume"].min()

0

In [47]:
np.log(data["Volume"].max() + 1e-8) 

9.869413572202895

In [10]:
def prCyan(skk): print("\033[96m {}\033[00m" .format(skk))

def uniques(category):    
    category_counts = data[category].value_counts()
    print("The number of unique values in column \"" + str(category) + "\" is: ", end = '')
    prCyan(str(len(category_counts)))
    print(category_counts.max())
    return category_counts

In [11]:
uniques("timeseries")

The number of unique values in column "timeseries" is:  204536
36


1         36
136350    36
136352    36
136353    36
136354    36
          ..
68182     36
68183     36
68184     36
68185     36
204536    36
Name: timeseries, Length: 204536, dtype: int64

In [28]:
data.to_csv('final_preprocessed_Data.csv')

In [29]:
data = pd.read_csv('./final_preprocessed_Data.csv')

In [31]:
data

,Unnamed: 0,Plant,Material,Volume,Date,SalesTargetGroup,time_idx,month,log_volume,avg_volume_by_material,avg_volume_by_plant,timeseries
0,0,5M01,AI15570013TARCA20,0,2019-02-01,510,0,2,-18.420681,0.894737,4.139019,1
1,1,5M01,AI15570013TARCA20,0,2019-03-01,510,1,3,-18.420681,2.631579,5.979067,1
2,2,5M01,AI15570013TARCA20,0,2019-04-01,510,2,4,-18.420681,0.000000,3.668722,1
3,3,5M01,AI15570013TARCA20,0,2019-05-01,510,3,5,-18.420681,0.000000,3.691088,1
4,4,5M01,AI15570013TARCA20,0,2019-06-01,510,4,6,-18.420681,0.421053,4.763329,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7363291,17180971,5V21,TT38652264TT3RTA0,0,2021-09-01,5A4,31,9,-18.420681,0.000000,2.287393,204536
7363292,17180972,5V21,TT38652264TT3RTA0,0,2021-10-01,5A4,32,10,-18.420681,0.000000,2.234502,204536
7363293,17180973,5V21,TT38652264TT3RTA0,0,2021-11-01,5A4,33,11,-18.420681,0.000000,3.513744,204536
7363294,17180974,5V21,TT38652264TT3RTA0,0,2021-12-01,5A4,34,12,-18.420681,0.000000,1.901611,204536


In [48]:
data['Plant'].isnull().sum()

0

In [49]:
data[data['avg_volume_by_plant'].isnull() == True]

,Plant,Material,Volume,Date,SalesTargetGroup,time_idx,month,log_volume,avg_volume_by_material,avg_volume_by_plant,timeseries


In [41]:
data = data[data['Material'].notna()]

In [42]:
data[data['avg_volume_by_plant'].isnull() == True]

,Unnamed: 0,Plant,Material,Volume,Date,SalesTargetGroup,time_idx,month,log_volume,avg_volume_by_material,avg_volume_by_plant,timeseries
